In [15]:
import numpy as np
import pandas as pd
import sqlite3
from annoy import AnnoyIndex
import requests
import os
import base64

In [21]:
%load_ext autoreload
%autoreload 2

### Data Preparation

- Ran data_preparation.sql on spotify.sqlite and add 'tracks_info' table to the sql database

# Test run with subset of the dataset (10,000 tracks)

In [2]:
data_path = 'data/spotify.sqlite'
conn = sqlite3.connect(data_path)
# conn.text_factory = lambda b: b.decode(errors = 'ignore')
cursor = conn.cursor()

In [3]:
query = '''
    SELECT *
    FROM tracks_info ti
'''
cursor.execute(query)
print("Query executed")

Query executed


In [4]:
index = AnnoyIndex(12, metric='angular')
index.set_seed(12)

In [5]:
for i in range(0, 8740043):
    batch = cursor.fetchmany(1)[0][4:]
    index.add_item(i, batch)
    if i == 10000:
        break

In [6]:
index.build(10, n_jobs=-1)

True

In [7]:
index.save('230329-test-10000_tracks.ann')

True

# Get audio features of a song from Spotify

In [22]:
a = os.environ.get('CLIENT_IDS')

In [23]:
a

In [16]:
    auth_url = 'https://accounts.spotify.com/api/token'
    param = {'grant_type': 'client_credentials'}
    auth_headers = {
        'Authorization': f"Basic {base64.b64encode(os.environ.get('CLIENT_ID'))}:{base64.b64encode(os.environ.get('CLIENT_SECRET'))}",
        'Content-Type': 'application/x-www-form-urlencoded'
    }

TypeError: a bytes-like object is required, not 'NoneType'

In [9]:
def search_song(track=None, artist=None, album=None):
    # get credentials
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_headers = {
        'Authorization': f"Basic {base64.b64encode(CLIENT_ID)}:{base64.b64encode(CLIENT_SECRET)}",
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    q = ''
    if track:
        q += "track:{track}"
    params = {
        'track': track, 
        'artist': artist,
        'album': album
    }
    url = 'https://api.spotify.com/v1/search'
    response = requests.get(url).json()

In [ ]:
search_song(track='Hello')